# Keras Example

Using an SMS Spam data set (slightly modified) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data set is a collection of 5574 SMS messages that have been labeled as ham or spam. The file is a tab-delimited file with the first column the label and the second the message content. I edited the data set to remove some unwanted columns and add headings. 



In [16]:
# some necessary packages
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
import pandas as pd

# set seed for reproducibility
np.random.seed(1234)

In [1]:
df = pd.read_csv('sms-spam.csv', header=0, usecols=[1,2], encoding='latin-1')
print('rows and columns:', df.shape)
print(df.head())

rows and columns: (4837, 2)
   spam                                               text
0     0  Go until jurong point, crazy.. Available only ...
1     0                      Ok lar... Joking wif u oni...
2     1  Free entry in 2 a wkly comp to win FA Cup fina...
3     0  U dun say so early hor... U c already then say...
4     0  Nah I don't think he goes to usf, he lives aro...


In [7]:
# split df into train and test
i = np.random.rand(len(df)) < 0.8
train = df[i]
test = df[~i]
print("train data size: ", train.shape)
print("test data size: ", test.shape)

train data size:  (3863, 2)
test data size:  (974, 2)


In [19]:
# set up X and Y
num_labels = 2
vocab_size = 25000
batch_size = 100

# fit the tokenizer on the training data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train.text)

x_train = tokenizer.texts_to_matrix(train.text, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test.text, mode='tfidf')

encoder = LabelEncoder()
encoder.fit(train.spam)
y_train = encoder.transform(train.spam)
y_test = encoder.transform(test.spam)

# check shape
print("train shapes:", x_train.shape, y_train.shape)
print("test shapes:", x_test.shape, y_test.shape)
print("test first five labels:", y_test[:5])


train shapes: (3863, 25000) (3863,)
test shapes: (974, 25000) (974,)
test first five labels: [0 0 1 1 0]


In [25]:
# fit Keras model
model = Sequential()
model.add(Dense(32, input_dim=vocab_size, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 32)                800032    
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 33        
Total params: 800,065
Trainable params: 800,065
Non-trainable params: 0
_________________________________________________________________
Train on 3476 samples, validate on 387 samples
Epoch 1/30
3476/3476 [==============================] - 3s - loss: 0.5894 - acc: 0.7621 - val_loss: 0.4307 - val_acc: 0.9354
Epoch 2/30
3476/3476 [==============================] - 1s - loss: 0.3137 - acc: 0.9497 - val_loss: 0.2250 - val_acc: 0.9716
Epoch 3/30
3476/3476 [==============================] - 1s - loss: 0.1523 - acc: 0.9807 - val_loss: 0.1265 - val_acc: 0.9845
Epoch 4/30
3476/3476 [==============================] - 1s - loss: 0.0789 - acc: 0.9942 - val_loss: 0.0861 - v

In [26]:
# evaluate
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Accuracy: ', score[1])

900/974 [==========================>...] - ETA: 0sAccuracy:  0.980492812285


In [27]:
print(score)

[0.10145307583944753, 0.98049281228494345]


In [28]:
# get predictions so we can calculate more metrics
pred = model.predict_classes(x_test)
print(pred[:5])

928/974 [===========================>..] - ETA: 0s[[0]
 [0]
 [1]
 [1]
 [0]]


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('accuracy score: ', accuracy_score(y_test, pred))
print('precision score: ', precision_score(y_test, pred))
print('recall score: ', recall_score(y_test, pred))
print('f1 score: ', f1_score(y_test, pred))

accuracy score:  0.980492813142
precision score:  1.0
recall score:  0.848
f1 score:  0.917748917749


This was by far the most accurate. Much more work could be done modifying the network topology. 